# PROPERTY BASED TESTING 
# IN PYTHON WITH HYPOTHESIS
- - - -
### DANIEL BRADBURN

##### ⚛ example based testing

##### ⚛ fuzz and property based testing

##### ⚛ hypothesis

##### ⚛ rule based stateful testing 

##### ⚛ django

## Example Based Testing

* say we have a run length encoding function
* we encode a string as characters and the number of consective occurences of that character
* let's just test this out with something simple 


In [ ]:
def encode(input_string):
    count = 1
    prev = ''
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                lst.append((prev, count))
            count = 1
            prev = character
        else:
            count += 1
    else:
        lst.append((character, count))
    return lst

In [ ]:
encode('hellllllo')

* and we also have a decode function which reconstructs the string
* let's just check this function, let's use the output from the encode

In [ ]:
def decode(lst):
    return ''.join(c * n for c, n in lst)

In [ ]:
decode([('h', 1), ('e', 1), ('l', 2), ('o', 1)])

* but it's probably best to formalize this in a unit test
* I'm using pytest here, but you could use unittest or your favourite test runner, the principal is the same

In [ ]:
def test_run_length_encode_decode():
    input_data = 'hello'
    assert decode(encode(input_data)) == input_data

* Looks good, tests are passing, everyone is happy.

In [12]:
import pytest

def pytest_run(test_name):
    pytest.cmdline.main(['.', '-k', test_name])
    
pytest_run('test_run_length_encode_decode')

================================================= test session starts =================================================
platform linux2 -- Python 2.7.6, pytest-2.9.2, py-1.4.31, pluggy-0.3.1
rootdir: /home/moagstar/Projects/property-based-testing, inifile: 
plugins: hypothesis-3.4.2
collected 0 items

============================================ no tests ran in 0.01 seconds =============================================


* But that's just one test case
* Maybe some other input might cause the test to fail?
* Let's parameterize the test

In [13]:
@pytest.mark.parametrize('input_data', [
    'hello',
    'Hello, World',
    'django',
    'uhhhmm...',
])
def test_run_length_encode_decode_parameterized(input_data):
    assert decode(encode(input_data)) == input_data

* Still all tests are passing, I'm starting to feel a bit more confident
* However, I'm not very creative, as you can see I struggled to come up with 4 test cases
* This is of course a simple example, 
* As our models become more complex, our ability to come up with good test cases them diminishes

In [ ]:
pytest_run('test_run_length_encode_decode_parameterized')

* So one tool we can employ is fuzz testing
* We try to find a falsifying example by throwing a bunch of random data at our test

## Fuzz testing

* So this is a simple fuzzing strategy we can use
* Basically we generate a bunch of random strings and give it to the test function

In [ ]:
import random, string

random.seed(0)

rand_lower = lambda: random.choice(string.lowercase)
rand_range = lambda: range(int(random.random() * 10))

def randomwords(n):
    return [''.join(rand_lower() for i in rand_range())
            for n in range(n)]

@pytest.mark.parametrize('input_data', randomwords(5))
def test_run_length_encode_decode_fuzzed(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
pytest_run('test_run_length_encode_decode_fuzzed')

* Looks good, everyting is still passing
* But something bugs me about this kind of fuzzing
* Perhaps I just didn't plug the right parameters into the fuzzer to find a failing example?
* This is where property based testing can help improve on fuzzing
* Let's take a closer look at property based testing

In [ ]:
@pytest.mark.parametrize('input_data', randomwords(10))
def test_run_length_encode_decode_fuzzed_fails(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
pytest_run('test_run_length_encode_decode_fuzzed_fails')

## Property Based testing

In [ ]:
from hypothesis import strategies as st
from hypothesis import given

@given(st.text())
def test_run_length_encode_decode_hypothesis(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
pytest_run('test_run_length_encode_decode_hypothesis')

## built in (and building) strategies

In [ ]:
st.integers(min_value=-100, max_value=100).example()

In [ ]:
st.text(st.characters(
        min_codepoint=64,
        max_codepoint=127)).example()

In [ ]:
st.lists(st.integers()).example()

In [ ]:
@st.composite
def composite_strategy(draw):
    # TODO : More interesting composite strategy
    return draw(st.one_of(st.integers(), st.text()))

composite_strategy().example()

In [ ]:
# TODO: json (recursive)

## rule based stateful testing 

In [ ]:
class Heap(list):
    
    def push(self, value):
        self += [value]
        index = len(self) - 1
        while index > 0:
            parent = (index - 1) // 2
            if self[parent] > self[index]:
                self[parent], self[index] = self[index], self[parent]
                index = parent
            else:
                break
        return self

In [ ]:
heap = Heap()
heap.push(99)
heap.push(0)

In [ ]:
heap = Heap()

In [ ]:
@given(st.lists(st.integers()))
def test_pop_in_sorted_order(lst):
    heap = Heap()
    map(heap.push, lst)
    r = [heap.pop() for _ in range(len(heap))]
    assert r == sorted(lst)

In [ ]:
pytest_run('test_pop_in_sorted_order')

In [ ]:
class HeapFixed(Heap):
    
    def pop(self):
        if len(self) == 0: raise ValueError("Empty self")
        if len(self) == 1: return super(HeapFixed, self).pop()
        result = self[0]
        self[0] = super(HeapFixed, self).pop()
        index = 0
        while index * 2 + 1 < len(self):
            children = [index * 2 + 1, index * 2 + 2]
            children = [i for i in children if i < len(self)]
            assert children
            children.sort(key=lambda x: self[x])
            for c in children:
                if self[index] > self[c]:
                    self[index], self[c] = self[c], self[index]
                    index = c
                    break
            else:
                break
        return result

In [ ]:
@given(st.lists(st.integers()))
def test_pop_in_sorted_order_fixed(lst):
    heap = HeapFixed()
    map(heap.push, lst)
    r = [heap.pop() for _ in range(len(heap))]
    assert r == sorted(lst)

In [ ]:
pytest_run('test_pop_in_sorted_order_fixed')

In [ ]:
def heapmerge(heap1, heap2):
    
    result = []
    i, j = 0, 0
    
    while i < len(heap1) and j < len(heap2):
        if heap1[i] <= heap2[j]:
            result.append(heap1[i])
            i += 1
        else:
            result.append(heap2[j])
            j += 1
            
    result.extend(heap1[i:])
    result.extend(heap2[j:])
    
    return result

In [ ]:
from hypothesis.stateful import rule 
from hypothesis.stateful import RuleBasedStateMachine

class HeapMachine(RuleBasedStateMachine):
    
    Heaps = Bundle('heaps')

    @rule(target=Heaps)
    def newheap(self): 
        return HeapFixed()

    @rule(heap=Heaps, value=integers())
    def push(self, heap, value): 
        heap.push(value)

    @rule(heap=Heaps.filter(bool))
    def pop(self, heap):
        assert min(heap) == heap.pop()

    @rule(target=Heaps, heap1=Heaps, heap2=Heaps)
    def merge(self, heap1, heap2):
        return heapmerge(heap1, heap2)

In [ ]:
# django